In [ ]:
# valence
# activity
# arousal 

# group by user
# find last day
# aggregate all except last day
# add variable for num days agged / adjust num days available

In [2]:
import pandas as pd
from datetime import datetime, timedelta

In [7]:
df = pd.read_csv("../dataset_mood_smartphone_clean.csv")

In [9]:
# sort values by time and reset index so it can be used for filtering

df = df.sort_values(['id','time'])
df.reset_index(drop=True,inplace=True)
df.head()

,id,time,variable,value,date,hour
0,AS14.01,2014-02-17 12:04:42.394,call,1.0,2014-02-17,12
1,AS14.01,2014-02-17 18:28:25.520,call,1.0,2014-02-17,18
2,AS14.01,2014-02-18 09:29:51.257,call,1.0,2014-02-18,9
3,AS14.01,2014-02-19 14:43:30.575,call,1.0,2014-02-19,14
4,AS14.01,2014-02-19 17:29:10.378,call,1.0,2014-02-19,17


In [10]:
# seperate out the target variable 
# target = the last day of mood measurment

target = df[df['variable']=='mood']
target = target.groupby(['id','date']).mean().reset_index()
target = target.groupby(['id']).nth(-1).reset_index()

In [11]:
num_days = 5
#add option to agg by hour?
users = list(df.id.unique())

In [12]:
# loop through id groups and extract num_days worth of day not including target day

AggregatedTime=[]
for i in users:
    # filter on id subset
    # remove last day of mood records 
    # reduce remaining data to previous num_days
    
    AggTime = df[df['id']==i]
    #AggTime.drop(AggTime.tail(1).index,inplace=True)
    targetIndex = AggTime['variable'].where(AggTime['variable']=='mood').last_valid_index()
    end_date = AggTime.loc[targetIndex].date
    start_date = str((datetime.fromisoformat(end_date) - timedelta(days=num_days)).date())
    date_range = [str(i.date()) for i in pd.date_range(start_date, end_date)]
    del date_range[-1]
    AggTime=  AggTime[AggTime['date'].isin(date_range)]
    AggregatedTime.append(AggTime)

In [13]:
AggregatedTime[0]

,id,time,variable,value,date,hour
19243,AS14.01,2014-04-29 00:00:00.000,activity,0.000,2014-04-29,0
19244,AS14.01,2014-04-29 00:02:50.125,appCat.builtin,19.316,2014-04-29,0
19245,AS14.01,2014-04-29 00:02:50.153,screen,19.314,2014-04-29,0
19246,AS14.01,2014-04-29 01:00:00.000,activity,0.000,2014-04-29,1
19247,AS14.01,2014-04-29 02:00:00.000,activity,0.000,2014-04-29,2
...,...,...,...,...,...,...
21620,AS14.01,2014-05-03 23:14:47.029,screen,75.131,2014-05-03,23
21621,AS14.01,2014-05-03 23:14:55.105,appCat.communication,67.007,2014-05-03,23
21622,AS14.01,2014-05-03 23:16:08.996,appCat.communication,7.129,2014-05-03,23
21623,AS14.01,2014-05-03 23:16:09.005,screen,9.464,2014-05-03,23


In [298]:
aDF = pd.concat(AggregatedTime)

In [330]:
# check for missing values in num_days range

valuesCheck = aDF.groupby(['id']).variable.value_counts().unstack(fill_value=0).reset_index()

In [332]:
valuesCheck[valuesCheck['id']=='AS14.01']

variable,id,activity,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,circumplex.arousal,circumplex.valence,mood,screen,sms
0,AS14.01,120,655,581,75,90,2,1,29,73,19,1,45,0,8,24,21,25,608,5


In [334]:
aDF[(aDF['id']=='AS14.01')&(aDF['variable']=='mood')]

,id,variable,value,date
19254,AS14.01,mood,6.0,2014-04-29
19341,AS14.01,mood,6.0,2014-04-29
19442,AS14.01,mood,7.0,2014-04-29
19519,AS14.01,mood,8.0,2014-04-29
19591,AS14.01,mood,8.0,2014-04-29
19666,AS14.01,mood,8.0,2014-04-30
19710,AS14.01,mood,7.0,2014-04-30
19903,AS14.01,mood,8.0,2014-04-30
19915,AS14.01,mood,8.0,2014-04-30
19994,AS14.01,mood,8.0,2014-04-30


In [335]:
# check that we have the expected number of dates
aDF['date'][aDF['id']=='AS14.01'].unique()

array(['2014-04-29', '2014-04-30', '2014-05-01', '2014-05-02',
       '2014-05-03'], dtype=object)

In [300]:
# drop unwanted columns

aDF = aDF[['id', 'variable', 'value', 'date']]

In [301]:
# split data into to sum and average

average = ['mood','circumplex.arousal','circumplex.valence','activity']
df_av = aDF[aDF['variable'].isin(average)]
df_sum = aDF[~aDF['variable'].isin(average)]

In [302]:
df_sum =df_sum.groupby(['id','variable']).sum().unstack().reset_index().sort_values(['id']).fillna(0)
df_av =df_av.groupby(['id','variable']).mean().unstack().reset_index().sort_values(['id']).fillna(0)
data = df_sum.merge(df_av,how='outer', on=["id"]).fillna(0)

In [303]:
data.columns=data.columns.get_level_values(1)

In [304]:
data.columns

Index(['', 'appCat.builtin', 'appCat.communication', 'appCat.entertainment',
       'appCat.finance', 'appCat.game', 'appCat.office', 'appCat.other',
       'appCat.social', 'appCat.travel', 'appCat.unknown', 'appCat.utilities',
       'appCat.weather', 'call', 'screen', 'sms', 'activity',
       'circumplex.arousal', 'circumplex.valence', 'mood'],
      dtype='object', name='variable')

In [305]:
cols = ['id', 'appCat.builtin', 'appCat.communication',
       'appCat.entertainment', 'appCat.finance', 'appCat.game',
       'appCat.office', 'appCat.other', 'appCat.social', 'appCat.travel',
       'appCat.unknown', 'appCat.utilities', 'appCat.weather', 'call',
       'screen', 'sms', 'activity', 'circumplex.arousal', 'circumplex.valence',
       'mood']
data.columns=cols

In [306]:
# checking that variable values haven't changed because of the merge
df_av = aDF[aDF['variable'].isin(agg)]
print(df_av['value'][df_av['variable'] == 'activity'].sum())
print(data['activity'].sum())

246.2977225349647
2.7483659151546163


In [217]:
aDF.head()

,id,variable,value,date
19243,AS14.01,activity,0.000,2014-04-29
19244,AS14.01,appCat.builtin,19.316,2014-04-29
19245,AS14.01,screen,19.314,2014-04-29
19246,AS14.01,activity,0.000,2014-04-29
19247,AS14.01,activity,0.000,2014-04-29


In [216]:
data.head()

,id,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,screen,sms,activity,circumplex.arousal,circumplex.valence,mood
0,AS14.01,9559.682,28042.367,5653.099,3062.187,333.305,28.098,343.697,6396.461,458.411,30.102,928.988,0.000,8.0,55840.403998,5.0,0.087991,-0.291667,0.952381,7.680000
1,AS14.02,0.000,0.000,0.000,0.000,0.000,0.000,0.000,106.887,0.000,0.000,0.000,0.000,39.0,13.080000,6.0,0.081835,-0.666667,-0.166667,5.666667
2,AS14.03,1598.930,4145.468,12209.611,56.467,0.000,14.902,1055.128,0.000,252.664,209.212,46.399,0.000,23.0,22151.371000,2.0,0.105415,-0.947368,1.052632,7.500000
3,AS14.05,4203.526,20346.719,1018.870,0.000,0.000,0.000,816.424,2523.125,0.000,647.470,87.530,0.000,3.0,29245.021996,12.0,0.044154,-0.130435,1.000000,6.913043
4,AS14.06,2021.525,14004.311,4517.613,0.000,0.000,419.788,866.989,9215.704,125.455,93.997,0.000,8.026,3.0,28065.064999,3.0,0.140943,-0.687500,0.400000,6.937500


In [307]:
data.isna().sum()

id                      0
appCat.builtin          0
appCat.communication    0
appCat.entertainment    0
appCat.finance          0
appCat.game             0
appCat.office           0
appCat.other            0
appCat.social           0
appCat.travel           0
appCat.unknown          0
appCat.utilities        0
appCat.weather          0
call                    0
screen                  0
sms                     0
activity                0
circumplex.arousal      0
circumplex.valence      0
mood                    0
dtype: int64

In [314]:
target = target[['id','value']]
target.columns = ['id','target']

In [315]:
final_data = data.merge(target,how='left', on=["id"])

In [316]:
final_data

,id,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,appCat.other,appCat.social,appCat.travel,...,appCat.utilities,appCat.weather,call,screen,sms,activity,circumplex.arousal,circumplex.valence,mood,target
0,AS14.01,9559.682,28042.367,5653.099,3062.187,333.305,28.098,343.697,6396.461,458.411,...,928.988,0.000,8.0,55840.403998,5.0,0.087991,-0.291667,0.952381,7.680000,8.000000
1,AS14.02,0.000,0.000,0.000,0.000,0.000,0.000,0.000,106.887,0.000,...,0.000,0.000,39.0,13.080000,6.0,0.081835,-0.666667,-0.166667,5.666667,9.000000
2,AS14.03,1598.930,4145.468,12209.611,56.467,0.000,14.902,1055.128,0.000,252.664,...,46.399,0.000,23.0,22151.371000,2.0,0.105415,-0.947368,1.052632,7.500000,7.666667
3,AS14.05,4203.526,20346.719,1018.870,0.000,0.000,0.000,816.424,2523.125,0.000,...,87.530,0.000,3.0,29245.021996,12.0,0.044154,-0.130435,1.000000,6.913043,6.333333
4,AS14.06,2021.525,14004.311,4517.613,0.000,0.000,419.788,866.989,9215.704,125.455,...,0.000,8.026,3.0,28065.064999,3.0,0.140943,-0.687500,0.400000,6.937500,7.000000
5,AS14.07,5689.399,22981.564,2493.024,0.000,0.000,0.000,1336.624,3187.685,0.000,...,0.000,0.000,1.0,22599.390001,0.0,0.066914,-1.071429,0.285714,5.785714,5.500000
6,AS14.08,414.305,15386.069,3983.720,0.000,0.000,0.000,212.061,2521.066,0.000,...,249.938,0.000,1.0,13012.908000,0.0,0.103794,-0.076923,0.961538,7.000000,6.666667
7,AS14.12,11731.141,7584.300,6483.892,735.365,181.611,215.338,233.880,17709.498,1008.236,...,35.895,0.000,4.0,46980.612002,3.0,0.109481,-0.380952,0.263158,5.666667,6.000000
8,AS14.13,1730.311,24472.498,0.000,0.000,0.000,299.272,1198.272,1849.199,1251.600,...,0.000,0.000,0.0,30060.119000,0.0,0.252988,-1.136364,1.136364,7.954545,8.000000
9,AS14.14,1218.189,191.628,0.000,0.000,0.000,0.000,221.418,231.781,0.000,...,0.000,0.000,0.0,1811.447001,0.0,0.029535,0.235294,0.882353,6.705882,7.000000


In [318]:
final_data.to_csv("../dataset_mood_smartphone_model.csv",index=False)
